1) Imports and Config

In [1]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import Dict, List
import csv
import os
import uvicorn
import nest_asyncio
from threading import Thread
import ipytest
import httpx
import logging


CSV_FILE = "patients.csv"
BASE_URL = "http://127.0.0.1:8000"
ipytest.autoconfig()

2. Data Model

In [2]:
class Patient(BaseModel):
    patient_id: str
    age: int
    sex: str
    BMI: float
    diagnosis_code: str
    sample_taken: bool
    blood_pressure: str
    cholesterol_level: float

3. CSV Helper Functions

In [3]:
def load_patients_from_csv() -> Dict[str, Patient]:
    if not os.path.exists(CSV_FILE):
        return {}
    with open(CSV_FILE, newline='') as f:
        reader = csv.DictReader(f)
        data = {}
        for row in reader:
            row['age'] = int(row['age'])
            row['BMI'] = float(row['BMI'])
            row['sample_taken'] = row['sample_taken'].lower() == "true"
            row['cholesterol_level'] = float(row['cholesterol_level'])
            data[row['patient_id']] = Patient(**row)
        return data

def save_patients_to_csv():
    with open(CSV_FILE, mode='w', newline='', encoding='utf-8') as f:
        fieldnames = list(Patient.__fields__)  # Pydantic v1 compatibility
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for patient in patients.values():
            writer.writerow(patient.dict())
        print("Saving to:", os.path.abspath(CSV_FILE))

4. FastAPI Initialisation

In [4]:
app = FastAPI()

# 🔧 Allow cross-origin requests from anywhere (for local dev)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # or ["http://127.0.0.1"] to be stricter
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Configure logging
log_file = "api_requests.log"
logging.basicConfig(
    filename=log_file,
    filemode="a",  # Append mode
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

# Optional: also log to console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
console_handler.setFormatter(formatter)
logging.getLogger().addHandler(console_handler)


patients: Dict[str, Patient] = load_patients_from_csv()

5. API Endpoints

In [6]:
@app.get("/patients", response_model=List[Patient])
def get_all_patients():
    return list(patients.values())

@app.get("/patients/{patient_id}", response_model=Patient)
def get_patient(patient_id: str):
    if patient_id not in patients:
        raise HTTPException(status_code=404, detail="Patient not found")
    return patients[patient_id]

@app.post("/patients", response_model=Patient, status_code=201)
def create_patient(patient: Patient):
    if patient.patient_id in patients:
        logging.warning(f"Creation failed: Patient {patient.patient_id} already exists.")
        raise HTTPException(status_code=400, detail="Patient already exists")        
    patients[patient.patient_id] = patient
    save_patients_to_csv()        
    logging.info(f"Patient created: {patient.patient_id}")
    return patient

@app.put("/patients/{patient_id}", response_model=Patient)
def update_patient(patient_id: str, updated: Patient):
    if patient_id not in patients:        
        logging.warning(f"Update failed: Patient {patient_id} not found.")
        raise HTTPException(status_code=404, detail="Patient not found")
    patients[patient_id] = updated
    save_patients_to_csv()
    logging.info(f"Patient updated: {patient_id}")
    return updated

@app.delete("/patients/{patient_id}", status_code=204)
def delete_patient(patient_id: str):
    if patient_id not in patients:    
        logging.warning(f"Delete failed: Patient {patient_id} not found.")
        raise HTTPException(status_code=404, detail="Patient not found")
    del patients[patient_id]
    save_patients_to_csv()
    logging.info(f"Patient deleted: {patient_id}")

6. Launch Server

In [7]:
nest_asyncio.apply()

def run():
    uvicorn.run(app, host="127.0.0.1", port=8000)

Thread(target=run, daemon=True).start()

INFO:     Started server process [22384]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:63823 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:63824 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:63823 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:63823 - "GET /openapi.json HTTP/1.1" 200 OK


7. Automation Tests

In [8]:
# Test 1: Get all patients
def test_get_all_patients():
    response = httpx.get(f"{BASE_URL}/patients")
    assert response.status_code == 200
    assert isinstance(response.json(), list)

# Test 2: Get a valid patient by ID
def test_get_valid_patient(patient_id: str):
    response = httpx.get(f"{BASE_URL}/patients/{patient_id}")
    assert response.status_code == 200
    assert response.json()["patient_id"] == patient_id

# Test 3: Get an invalid (nonexistent) patient
def test_get_invalid_patient(patient_id: str):
    response = httpx.get(f"{BASE_URL}/patients/{patient_id}")
    assert response.status_code == 404

# Test 4: Create a new patient
def test_create_new_patient(patient_id: str):
    new_patient = {
        "patient_id": patient_id,
        "age": 55,
        "sex": "Male",
        "BMI": 28.5,
        "diagnosis_code": "I90",
        "sample_taken": True,
        "blood_pressure": "125/85",
        "cholesterol_level": 5.5
    }
    response = httpx.post(f"{BASE_URL}/patients", json=new_patient)
    assert response.status_code == 201
    assert response.json()["patient_id"] == patient_id

# Test 5: Update an existing patient
def test_update_patient(patient_id: str, new_age: int):
    updated_data = {
        "patient_id": patient_id,
        "age": new_age,
        "sex": "Male",
        "BMI": 29.0,
        "diagnosis_code": "I91",
        "sample_taken": False,
        "blood_pressure": "120/80",
        "cholesterol_level": 5.0
    }
    response = httpx.put(f"{BASE_URL}/patients/{patient_id}", json=updated_data)
    assert response.status_code == 200
    assert response.json()["age"] == new_age

# Test 6: Delete a patient
def test_delete_patient(patient_id: str):
    response = httpx.delete(f"{BASE_URL}/patients/{patient_id}")
    assert response.status_code == 204
    # Confirm deletion
    check = httpx.get(f"{BASE_URL}/patients/{patient_id}")
    assert check.status_code == 404

In [ ]:
test_get_all_patients()
test_get_valid_patient("PAT1002")
test_get_invalid_patient("PAT10000")
test_create_new_patient("TEST001")
test_update_patient("TEST001", 50)
test_delete_patient("TEST001")

In [9]:
test_get_all_patients()

INFO:     127.0.0.1:63851 - "GET /patients HTTP/1.1" 200 OK


2025-06-16 10:40:11,842 - INFO - HTTP Request: GET http://127.0.0.1:8000/patients "HTTP/1.1 200 OK"


In [10]:
test_get_valid_patient("PAT1001")

INFO:     127.0.0.1:63857 - "GET /patients/PAT1001 HTTP/1.1" 200 OK


2025-06-16 10:40:16,036 - INFO - HTTP Request: GET http://127.0.0.1:8000/patients/PAT1001 "HTTP/1.1 200 OK"


In [11]:
test_get_invalid_patient("PAT10000")

INFO:     127.0.0.1:63864 - "GET /patients/PAT10000 HTTP/1.1" 404 Not Found


2025-06-16 10:40:21,846 - INFO - HTTP Request: GET http://127.0.0.1:8000/patients/PAT10000 "HTTP/1.1 404 Not Found"


In [12]:
test_create_new_patient("TEST001")

2025-06-16 10:40:26,384 - INFO - Patient created: TEST001


Saving to: C:\Users\mfbx2jrj\UKBiobank-Interview\patients.csv
INFO:     127.0.0.1:63870 - "POST /patients HTTP/1.1" 201 Created


2025-06-16 10:40:26,384 - INFO - HTTP Request: POST http://127.0.0.1:8000/patients "HTTP/1.1 201 Created"


In [13]:
test_update_patient("TEST001", 50)

2025-06-16 10:40:39,982 - INFO - Patient updated: TEST001


Saving to: C:\Users\mfbx2jrj\UKBiobank-Interview\patients.csv
INFO:     127.0.0.1:63889 - "PUT /patients/TEST001 HTTP/1.1" 200 OK


2025-06-16 10:40:39,986 - INFO - HTTP Request: PUT http://127.0.0.1:8000/patients/TEST001 "HTTP/1.1 200 OK"


In [14]:
test_delete_patient("TEST00111")

2025-06-16 10:40:45,929 - WARNING - Delete failed: Patient TEST00111 not found.


INFO:     127.0.0.1:63898 - "DELETE /patients/TEST00111 HTTP/1.1" 404 Not Found


2025-06-16 10:40:45,929 - INFO - HTTP Request: DELETE http://127.0.0.1:8000/patients/TEST00111 "HTTP/1.1 404 Not Found"


AssertionError: assert 404 == 204
 +  where 404 = <Response [404 Not Found]>.status_code

INFO:     127.0.0.1:64112 - "GET /patients HTTP/1.1" 200 OK
INFO:     127.0.0.1:64114 - "OPTIONS /patients HTTP/1.1" 200 OK


...

Saving to: C:\Users\mfbx2jrj\UKBiobank-Interview\patients.csv
INFO:     127.0.0.1:64594 - "POST /patients HTTP/1.1" 201 Created
INFO:     127.0.0.1:64594 - "GET /patients HTTP/1.1" 200 OK
INFO:     127.0.0.1:64616 - "GET /patients HTTP/1.1" 200 OK
INFO:     127.0.0.1:64616 - "OPTIONS /patients/TEST999 HTTP/1.1" 200 OK


2025-06-16 10:47:37,898 - INFO - Patient updated: TEST999


Saving to: C:\Users\mfbx2jrj\UKBiobank-Interview\patients.csv
INFO:     127.0.0.1:64616 - "PUT /patients/TEST999 HTTP/1.1" 200 OK
INFO:     127.0.0.1:64616 - "GET /patients HTTP/1.1" 200 OK
INFO:     127.0.0.1:64628 - "GET /patients HTTP/1.1" 200 OK
INFO:     127.0.0.1:64628 - "OPTIONS /patients/TEST999 HTTP/1.1" 200 OK


2025-06-16 10:47:45,043 - INFO - Patient deleted: TEST999


Saving to: C:\Users\mfbx2jrj\UKBiobank-Interview\patients.csv
INFO:     127.0.0.1:64628 - "DELETE /patients/TEST999 HTTP/1.1" 204 No Content
INFO:     127.0.0.1:64628 - "GET /patients HTTP/1.1" 200 OK
